In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Underweight"
cohort = "GSE50982"

# Input paths
in_trait_dir = "../../input/GEO/Underweight"
in_cohort_dir = "../../input/GEO/Underweight/GSE50982"

# Output paths
out_data_file = "../../output/preprocess/Underweight/GSE50982.csv"
out_gene_data_file = "../../output/preprocess/Underweight/gene_data/GSE50982.csv"
out_clinical_data_file = "../../output/preprocess/Underweight/clinical_data/GSE50982.csv"
json_path = "../../output/preprocess/Underweight/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE50982_family.soft.gz', 'GSE50982_series_matrix.txt.gz']
Identified SOFT files: ['GSE50982_family.soft.gz']
Identified matrix files: ['GSE50982_series_matrix.txt.gz']

Background Information:
!Series_title	"Cavin-3 Dictates the Balance Between ERK and Akt Signaling"
!Series_summary	"Cavin-3 is a tumor suppressor protein of unknown function. Using a combination of in vivo knockout and in vitro gain/loss of function approaches, we show that cavin-3 dictates the balance between ERK and Akt signaling. Loss of cavin-3 increases Akt signaling at the expense of ERK, while gain of cavin-3 increases ERK signaling at the expense Akt. Cavin-3 facilitates signal transduction to ERK by anchoring caveolae, a lipid-raft specialization that contains an ERK activation module, to the membrane skeleton of the plasma membrane. Loss of cavin-3 reduces the number of caveolae, thereby separating this ERK activation module from signaling receptors. Loss of cavin-3 promotes A

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to be studying gene expression
# changes related to Cavin-3 protein's effect on ERK and Akt signaling pathways
is_gene_available = True

# 2.1 Data Availability
# For trait (Underweight): 
# From the sample characteristics, we can infer underweight status from knockdown days
# Knockdown days information is available in row 1 of the characteristics dictionary
trait_row = 1

# For age: Age information is not available in this dataset
age_row = None

# For gender: Gender information is not available in this dataset
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert knockdown data to binary underweight status.
    
    Based on the background information, we can infer that longer 
    knockdown of Cavin-3 leads to cachexia (wasting syndrome),
    which can be considered as an underweight condition.
    
    8+ days of knockdown are considered as underweight (1)
    0-3 days of knockdown are considered normal weight (0)
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        knockdown_days = int(value)
        # Based on biological inference: longer knockdown leads to cachexia/underweight
        if knockdown_days >= 8:  # 8 or more days of knockdown
            return 1  # Underweight
        else:  # Less than 8 days of knockdown
            return 0  # Normal weight
    except (ValueError, TypeError):
        return None

def convert_age(value):
    # Age data is not available, but we need this function for the library
    return None

def convert_gender(value):
    # Gender data is not available, but we need this function for the library
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since the clinical data variable was used previously, we'll assume it's already loaded
if trait_row is not None:
    try:
        # Assuming clinical_data is already available from previous steps
        # If not, we need to store this information and continue with metadata
        
        # Use sample characteristics as clinical data
        # Create a DataFrame from the sample characteristics dictionary
        characteristics = {0: ['cell line: SV589'], 
                          1: ['knockdown (days): 0', 'knockdown (days): 3', 'knockdown (days): 8', 'knockdown (days): 15'], 
                          2: ['treatment: Serum Starved; no EGF', 'treatment: Serum Starved; 1h EGF', 
                              'treatment: Serum Starved; 3h EGF', 'treatment: no treatment']}
        
        # Create a clinical data DataFrame with empty samples
        # The geo_select_clinical_features function will handle the association between samples and characteristics
        clinical_data = pd.DataFrame()
        clinical_data['characteristics'] = pd.Series(characteristics)
        
        # Use the library function to extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features to CSV
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to: {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")
        # If clinical data processing fails, update the metadata
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=False
        )


Preview of clinical features:
{'characteristics': [nan]}
Clinical features saved to: ../../output/preprocess/Underweight/clinical_data/GSE50982.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data shape: (47323, 45)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers, they start with "ILMN_" which indicates they are Illumina 
# microarray probe IDs, not human gene symbols. These need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene mapping
# From the gene annotation preview, we can see:
# - 'ID' column contains the Illumina probe IDs (ILMN_*)
# - 'Symbol' column contains gene symbols

# 2. Get gene mapping dataframe
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'Symbol'  # Column containing gene symbols

# Extract mapping between probes and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 genes after mapping:")
print(gene_data.index[:10])


Gene mapping dataframe shape: (44837, 2)
Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}


Gene expression data shape after mapping: (21464, 45)
First 10 genes after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print(f"First few gene symbols after normalization: {list(gene_data.index[:10])}")

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in Step 2
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical features extracted:")
print(preview_df(clinical_features))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(preview_df(linked_data))

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")

# 6. Conduct quality check and save the cohort information
note = "This dataset contains data on 16p11.2 duplication carriers associated with underweight."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (20259, 45)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/Underweight/gene_data/GSE50982.csv
Clinical features extracted:
{'characteristics': [nan]}
Clinical data saved to ../../output/preprocess/Underweight/clinical_data/GSE50982.csv
Linked data shape: (46, 20260)
Linked data preview:


{'Underweight': [nan, nan, nan, nan, nan], 'A1BG': [nan, 230.2868, 238.90699999999998, 234.83589999999998, 236.6119], 'A1BG-AS1': [nan, 115.7004, 136.1917, 133.0038, 128.777], 'A1CF': [nan, 315.4479, 324.1434, 326.9045, 336.3676], 'A2M': [nan, 99.78751, 100.2313, 100.2132, 94.68452], 'A2ML1': [nan, 94.3905, 107.1121, 104.4166, 97.52838], 'A3GALT2': [nan, 220.3246, 225.5058, 214.4992, 204.40794], 'A4GALT': [nan, 160.7064, 240.7132, 220.7152, 163.5258], 'A4GNT': [nan, 116.5021, 121.6363, 121.9945, 123.2215], 'AAA1': [nan, 524.87845, 555.8584, 538.0862, 514.4675199999999], 'AAAS': [nan, 145.673, 176.9964, 180.3475, 153.893], 'AACS': [nan, 184.948, 274.0733, 288.8924, 219.36], 'AACSP1': [nan, 132.3505, 129.8966, 134.7291, 135.2525], 'AADAC': [nan, 106.4089, 108.2836, 112.9999, 106.6635], 'AADACL2': [nan, 108.3963, 108.0997, 107.609, 108.9314], 'AADACL3': [nan, 95.65421, 98.78402, 107.4066, 102.6192], 'AADACL4': [nan, 110.7212, 107.8824, 108.4992, 110.8268], 'AADAT': [nan, 512.40888, 583.21